<a href="https://colab.research.google.com/github/ayuksekdag/pyspark_learning/blob/main/pyspark1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP 

In [14]:
#step 1 check if everything is there
!ls

In [ ]:
# step 2: download spark engine
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
# step 3 download crimes files
!wget "https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD&bom=true&format=true&delimiter=%3B"
# rename file
!mv "rows.csv?accessType=DOWNLOAD&bom=true&format=true&delimiter=;" reported-crimes.csv


In [13]:
!ls

reported-crimes.csv  spark-2.3.1-bin-hadoop2.7
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


In [20]:
# step 4 set the environment
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

# step 5 check if spark is there
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate() 
print(sc)

#step 6 check if pyspark is there 
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
print(spark)


<SparkContext master=local[*] appName=pyspark-shell>


In [55]:
from pyspark.sql.functions import to_timestamp, col, lit
 
df = spark.read.options( \
                        delimiter=';' \
                        ,header='True' \
                        ,dateFormat = "yyyy-MM-dd HH:mm:ss a" \
                        ,inferSchema = "True"
                        ,nullValue = 'null' \
                        ) \
                        .csv('reported-crimes.csv') 
# df = spark.read.csv('reported-crimes.csv',header=True)
df.show(5)



+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true| 924|   

#START YOUR CODE 